In [ ]:
import io, os, ssl, ast, json, http, socket, queue, struct, sqlite3, threading, logging, datetime 
import flask, flask_basicauth, flask_cors  
#from IPython.display import clear_output

server_ip    = 'guoxiaokang.net'
img_port     = 54321
ctrl_port    = 54322
flask_port   = 54323
sensor_port  = 54324

with open('src/map.html', 'r') as fr: mapPage = fr.read()
with open('src/rainbow.jpg', 'rb') as fr: rainbow = fr.read() 
with open('secret/basicAuth.conf', 'r') as fr: basicAuth = fr.read()
basic_auth_username = basicAuth.split('\n')[0]
basic_auth_password = basicAuth.split('\n')[1]
server_cert_letsencrypt  = f'{os.environ["HOME"]}/openssl/fullchain.pem'  # secret of my apache and flask site
server_key_letsencrypt   = f'{os.environ["HOME"]}/openssl/privkey.pem'    # secret of my apache and flask site
server_cert_self_signed  = 'secret/server.crt'    # shared with both rover and driver for identity authentication of server 
server_key_self_signed   = 'secret/server.key'    
client_certs_self_signed = 'secret/clients.crt'   # for identity authentication of rover and driver

ssl_context_self_signed = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH, cafile=client_certs_self_signed)  
ssl_context_self_signed.verify_mode = ssl.CERT_REQUIRED
ssl_context_self_signed.load_cert_chain(certfile=server_cert_self_signed, keyfile=server_key_self_signed) 

In [ ]:
def getCommonName(sc):
    for i in sc.getpeercert()['subject']:
        if i[0][0]=='commonName':
            return i[0][1]
        
ctrl_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ctrl_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
ctrl_socket.bind((server_ip, ctrl_port))
ctrl_socket.listen(0)

In [ ]:
class ProactivelyEndConnection(Exception): pass
class ConnectionHasEnded(Exception): pass

def transceiver(clients): 
    sc_rover  = clients['rover' ]
    sc_driver = clients['driver']   
    print('Both ends connected with the Server ~')
    while True:
        try:
            data = sc_driver.recv(8) # struct.calcsize('<hi')   
        except socket.timeout:
            raise ProactivelyEndConnection('driver') 
        except struct.error as e:
            print('received nothing from driver side')
            raise ProactivelyEndConnection('driver') 
        except ConnectionResetError:
            print('rover ConnectionResetError')
            raise ConnectionHasEnded('driver') 
        try:
            sc_rover.send(data) # Send code and value to server 
        except socket.timeout:
            print('rover timeout')
            raise ProactivelyEndConnection('rover')
        except BrokenPipeError:
            print('rover BrokenPipeError')
            raise ConnectionHasEnded('rover') 
        except ConnectionResetError:
            print('rover ConnectionResetError')
            raise ConnectionHasEnded('rover') 

In [ ]:
clients = {}
#def ctrld():
if 1:
    while True:
        try:
            print(f"Waiting for incoming connection")
            c, a = ctrl_socket.accept() 
            sc = ssl_context_self_signed.wrap_socket(c, server_side=True)
            sc.settimeout(30)
            client_name = getCommonName(sc)
            print(f"Connection with {client_name} established ~")
            clients[client_name] = sc
            print(clients)
            if len(clients) == 2:
                transceiver(clients) 
        except ConnectionHasEnded:
            pass
        except ProactivelyEndConnection as e:
            client_name = e.__str__()               # name of broken secure socket
            old_sc = clients.pop(client_name)       # get the socket 
            old_sc.shutdown(socket.SHUT_RDWR)       # explicitly close the socket 
            old_sc.close()            
            print(f"Old connection from {client_name} is timed-out and closed") 
        
#threading.Thread(target=ctrld).start()

In [ ]:
raise

In [ ]:
def ctrd():
    l = struct.calcsize('<hi')
    while True: xbox.action(*struct.unpack('<hi', sc_driver.recv(l))) 
# this background thread will monitor axis event and modify variable axis in a real-time manner
threading.Thread(target=ctrd).start()

#### A standalone thread is responsible for image cache

In [ ]:
class MJPEG():
    def __init__(self):
        self.frame = rainbow # default image
        self.long  = struct.calcsize('<L')  
        threading.Thread(target=self.imgd).start() 

    def imgd(self): 
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s.bind((server_ip, img_port))
        s.listen(0)
        while True: 
            c, a = s.accept() 
            conn = ssl_context_self_signed.wrap_socket(c, server_side=True).makefile('rb')
            while True:
                try:
                    img_len, = struct.unpack('<L', conn.read(self.long)) 
                    self.frame = conn.read(img_len) 
                except Exception as e:
                    print(e, a)
                    break                    
mjpeg = MJPEG()

#### A class taking care of database operation
the newcomer need to retrieve all GPS tracking data and that's the reason of involving a database 

In [ ]:
q_in, q_out = queue.Queue(), queue.Queue()  
def dbd(): # avoid sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread.
    #db = sqlite3.connect(":memory:")
    try:
        os.remove("GPS.db")
    except:
        pass
    db = sqlite3.connect("GPS.db")
    cur = db.cursor() 
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", ('tracking',))
    if not cur.fetchone(): cur.execute("CREATE TABLE tracking (lat, lon, timestamp)") 
    while True:
        item = q_in.get()
        if isinstance(item, tuple):
            cur.execute("INSERT INTO tracking (lat, lon, timestamp) VALUES (?, ?, ?)", item) 
        else: 
            if item == 'newcomer':
                cur.execute("SELECT lon, lat, timestamp FROM tracking WHERE timestamp=(SELECT MIN(timestamp) FROM tracking)") 
            else:
                cur.execute("SELECT lon, lat, timestamp FROM tracking WHERE timestamp>=?", (float(item),))
            q_out.put(cur.fetchall())
        q_in.task_done()
        
threading.Thread(target=dbd).start() 

#### A standalone thread is responsible for sensor logging (plus GPS logging/insertion)

In [ ]:
class SENSOR():
    def __init__(self):
        self.data = {'N':0, 'x':0, 'y':0, 'z':0, 'focus':'true'}
        self.long  = struct.calcsize('<H')  
        threading.Thread(target=self.updaterd).start() 

    def updaterd(self):
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s.bind((server_ip, sensor_port))
        s.listen(0)
        while True: 
            c, a = s.accept() 
            conn = ssl_context_self_signed.wrap_socket(c, server_side=True).makefile('rb')
            while True:
                try:
                    data_len, = struct.unpack('<H', conn.read(self.long)) 
                    j = json.loads(conn.read(data_len))   
                    if 'P' in j: 
                        GPRMC = pynmea2.parse(j['P']) # http://aprs.gids.nl/nmea/
                        lon, lat = GPRMC.lon, GPRMC.lat
                        lat = float(lat[:2]) + float(lat[2:])/60
                        lon = float(lon[:3]) + float(lon[3:])/60
                        if GPRMC.lat_dir == 'S': lat *= -1 
                        if GPRMC.lon_dir == 'W': lon *= -1
                        timestamp = datetime.datetime.combine(GPRMC.datestamp, GPRMC.timestamp).timestamp() 
                        q_in.join()
                        q_in.put((lat, lon, timestamp)) 
                    if 'N' in j and 'a' in j:
                        self.data['N'] = 180 - int(j['N']) 
                        x, y, z = ast.literal_eval(j['a']) 
                        self.data['x'], self.data['y'], self.data['z'] = x/-20.48, y/20.48, z 
                except Exception as e:
                    print(e, a)
                    break
sensor = SENSOR()

In [ ]:
app = flask.Flask(__name__)
flask_cors.CORS(app) 
app.config['BASIC_AUTH_USERNAME'] = basic_auth_username
app.config['BASIC_AUTH_PASSWORD'] = basic_auth_password
basic_auth = flask_basicauth.BasicAuth(app)

#log = logging.getLogger('werkzeug')
#log.setLevel(logging.ERROR) 

@app.route("/")
@basic_auth.required
def home(): 
    return flask.render_template_string(mapPage, 
        ip_port       = f'{server_ip}:{flask_port}', 
        username      =  basic_auth_username, 
        password      =  basic_auth_password)  

@app.route("/stream.mjpg")
@basic_auth.required
def mjpg():    
    def generator(): 
        while True:  
            frame = mjpeg.frame 
            yield f'''--FRAME\r\nContent-Type: image/jpeg\r\nContent-Length: {len(frame)}\r\n\r\n'''.encode() 
            yield frame
    r = flask.Response(response=generator(), status=200)
    r.headers.extend({'Age':0, 'Content-Type':'multipart/x-mixed-replace; boundary=FRAME',
                      'Pragma':'no-cache', 'Cache-Control':'no-cache, private',}) 
    return r

RESPONSE = sensor.data 
@app.route("/sensor", methods=['GET'])
@basic_auth.required
def sensor():
    global RESPONSE   
    newest_timestamp_client_side = flask.request.args.get('timestamp') 
    q_in.join()  
    q_in.put(newest_timestamp_client_side)
    q_in.join() 
    llts = q_out.get() # lat, long, timestamp 
    if llts:
        RESPONSE.update({
            "rover": {
                "type": "Feature", 
                "geometry": {
                    "type": "Point",
                    "coordinates": llts[-1][:2] # long lat in GeoJson
                }, 
                "properties": {
                    "newestViewCenter": llts[-1][1::-1], # lat long in Google and Openstreetmap
                    "newestTimestampServerSide": llts[-1][2],
                    "homePoint": "false"
                },
            }
        })
        if newest_timestamp_client_side == 'newcomer': 
            RESPONSE['rover']['properties']['homePoint'] = llts[-1][1::-1]
    if len(llts) > 1: 
        RESPONSE.update({
            "trail": {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": [i[:2] for i in llts]
                }
            }
        })
    return flask.jsonify(RESPONSE)

@app.route('/axios.min.js', methods=['GET'])
def axios_js():
        return flask.send_file('src/axios.min.js', mimetype='application/javascript')
    
@app.route('/leaflet.rotatedMarker.js', methods=['GET'])
def rotatedMarker_js():
        return flask.send_file('src/leaflet.rotatedMarker.js', mimetype='application/javascript')
    
@app.route('/home.png', methods=['GET'])
def home_svg():
        return flask.send_file('src/home.png', mimetype='image/png')
    
@app.route('/arrow.png', methods=['GET'])
def arrow_svg():
        return flask.send_file('src/arrow.png', mimetype='image/png')

def flask_in_bg():
    app.run(server_ip, flask_port, ssl_context=(server_cert_letsencrypt, server_key_letsencrypt))
threading.Thread(target=flask_in_bg).start() 
#flask_in_bg()